In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [2]:
def load_data_x_y(x_path: str, y_path: str, do_label_flip: bool, to_one_hot: bool, normalize: bool = True):
    x = np.load(x_path)
    y = np.load(y_path)
    y_max = np.max(y)
    y_min = np.min(y)

    if do_label_flip:
        y = y_max - y + y_min
    if to_one_hot:
        y = tf.keras.utils.to_categorical(y, num_classes=y_max - y_min + 1)

    if normalize:
        x = x.astype(np.float32) / 255.

    return tf.data.Dataset.from_tensor_slices((x, y))

In [3]:
def data_augmentation(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=32.0 / 255.0)
    img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
    img = tf.clip_by_value(img, 0.0, 1.0)

    return img, label

In [4]:
with tf.device('/cpu:0'):
    ds = load_data_x_y('cifar10_x_train.npy', 'cifar10_y_train.npy', False, True, True).map(data_augmentation, num_parallel_calls=8)
    ds = ds.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
    test_ds = load_data_x_y('cifar10_x_test.npy', 'cifar10_y_test.npy', False, True, True)
    test_ds = test_ds.batch(128).prefetch(tf.data.AUTOTUNE)

2022-03-19 16:44:15.240281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-19 16:44:15.240546: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [5]:
model = tf.keras.models.load_model('/Users/maghsk/Gits/defl-hotstuff/benchmark/defl/data/init_model.h5')
# model = tf.keras.applications.ResNet50V2(input_shape=(32, 32, 3), classes=10, weights=None)

In [6]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    optimizer=tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-5),
)
model.summary()

Model: "init_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        36928     
                                                        

In [7]:
history = model.fit(ds, epochs=10, validation_data=test_ds, validation_freq=1)

Epoch 1/10


2022-03-19 16:44:17.009454: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-19 16:44:17.009984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - ETA: 0s - loss: 1.5350 - categorical_accuracy: 0.4415

2022-03-19 16:44:48.623296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 33s 20ms/step - loss: 1.5350 - categorical_accuracy: 0.4415 - val_loss: 1.1403 - val_categorical_accuracy: 0.5934
Epoch 2/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.1116 - categorical_accuracy: 0.6033 - val_loss: 0.9292 - val_categorical_accuracy: 0.6762
Epoch 3/10
1563/1563 [==============================] - 34s 22ms/step - loss: 0.9585 - categorical_accuracy: 0.6618 - val_loss: 0.8434 - val_categorical_accuracy: 0.6979
Epoch 4/10
1563/1563 [==============================] - 34s 21ms/step - loss: 0.8632 - categorical_accuracy: 0.6965 - val_loss: 0.7794 - val_categorical_accuracy: 0.7255
Epoch 5/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.8045 - categorical_accuracy: 0.7187 - val_loss: 0.7066 - val_categorical_accuracy: 0.7548
Epoch 6/10
1563/1563 [==============================] - 31s 20ms/step - loss: 0.7663 - categorical_accuracy: 0.7315 - val_loss: 0.7107 - val_categorical_accuracy